Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

load data and see features

In [ ]:
#load data
df = pd.read_csv("/content/drive/MyDrive/titanic_dataset.csv")
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [ ]:
#see features
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [ ]:
#look for missing values
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [ ]:
#fix missing value problem
df['Age'].fillna(df['Age'].median(), inplace=True) #fill with median
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True) #fill with mode
df['Cabin'].fillna('Unknown', inplace=True) # make a new class called unknown
# df.drop('Cabin', axis=1, inplace=True)

In [ ]:
#last look
df.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [ ]:
#wt cat do i have
catg = [var for var in df.columns if df[var].dtype=='O']
catg
#name m4 far2 awy

['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']

In [ ]:
#remove non affecting columns
df.drop('Name', axis=1, inplace=True)
df.drop('Ticket', axis=1, inplace=True)
df.drop('PassengerId', axis=1, inplace=True)


catg = [var for var in df.columns if df[var].dtype=='O']
catg

['Sex', 'Cabin', 'Embarked']

In [ ]:
#view if there any data not right for the column as simbol that is not null
for i in range(len(catg)):
  print(df[catg[i]].value_counts())
  print("##################")

Sex
male      577
female    314
Name: count, dtype: int64
##################
Cabin
Unknown        687
C23 C25 C27      4
G6               4
B96 B98          4
C22 C26          3
              ... 
E34              1
C7               1
C54              1
E36              1
C148             1
Name: count, Length: 148, dtype: int64
##################
Embarked
S    646
C    168
Q     77
Name: count, dtype: int64
##################


In [ ]:
#fetures and target split
x = df.drop(['Survived'], axis=1)
y = df['Survived']
x

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,male,22.0,1,0,7.2500,Unknown,S
1,1,female,38.0,1,0,71.2833,C85,C
2,3,female,26.0,0,0,7.9250,Unknown,S
3,1,female,35.0,1,0,53.1000,C123,S
4,3,male,35.0,0,0,8.0500,Unknown,S
...,...,...,...,...,...,...,...,...
886,2,male,27.0,0,0,13.0000,Unknown,S
887,1,female,19.0,0,0,30.0000,B42,S
888,3,female,28.0,1,2,23.4500,Unknown,S
889,1,male,26.0,0,0,30.0000,C148,C


In [ ]:
###############make cabin into letter classes

# x = pd.get_dummies(x,columns=x[catg[:8]].columns,dtype='int64')
x['Cabin_Class'] = x['Cabin'].apply(lambda x: x[0] if x != 'Unknown' else 'Unknown')

x.drop('Cabin', axis=1, inplace=True)

catg = ['Cabin_Class'] + catg
catg = [col for col in catg if col != 'Cabin']

##############3turn all to one hot encoded columns

x = pd.get_dummies(x, columns=catg, dtype='int64')
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# x[['Fare']] = scaler.fit_transform(x[['Fare']])
x

,Pclass,Age,SibSp,Parch,Fare,Cabin_Class_A,Cabin_Class_B,Cabin_Class_C,Cabin_Class_D,Cabin_Class_E,Cabin_Class_F,Cabin_Class_G,Cabin_Class_T,Cabin_Class_Unknown,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,22.0,1,0,7.2500,0,0,0,0,0,0,0,0,1,0,1,0,0,1
1,1,38.0,1,0,71.2833,0,0,1,0,0,0,0,0,0,1,0,1,0,0
2,3,26.0,0,0,7.9250,0,0,0,0,0,0,0,0,1,1,0,0,0,1
3,1,35.0,1,0,53.1000,0,0,1,0,0,0,0,0,0,1,0,0,0,1
4,3,35.0,0,0,8.0500,0,0,0,0,0,0,0,0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,2,27.0,0,0,13.0000,0,0,0,0,0,0,0,0,1,0,1,0,0,1
887,1,19.0,0,0,30.0000,0,1,0,0,0,0,0,0,0,1,0,0,0,1
888,3,28.0,1,2,23.4500,0,0,0,0,0,0,0,0,1,1,0,0,0,1
889,1,26.0,0,0,30.0000,0,0,1,0,0,0,0,0,0,0,1,1,0,0


In [ ]:
#train test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 42)

train and test


In [ ]:
#train the model
gauss = GaussianNB()
gauss.fit(x_train, y_train)

GaussianNB()

In [ ]:
#predict the test values
y_pred = gauss.predict(x_test)
y_pred

array([0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0])

In [ ]:
#see the accuracy
accuracy_score(y_test, y_pred)

0.7835820895522388

In [ ]:
#evaluation with accuracy, precision, recall and f1
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Accuracy: 0.7835820895522388
Precision: 0.7345132743362832
Recall: 0.7477477477477478
F1-score: 0.7410714285714286


In [ ]:
#preform cross validation to avoid over fitting
cv_scores = cross_val_score(gauss, x, y, cv=10)
print("Mean CV score:", cv_scores.mean())

Mean CV score: 0.7723754789272029
